In [ ]:
# medir tiempos
%load_ext autotime

![output](tiempo_secuencial.png)

## Paralelo pyspark 

In [1]:
# Modulo para encontrar pyspark
import findspark
findspark.init("/usr/local/spark/spark-3.1.1-bin-hadoop2.7")

In [2]:
# importamos pyspark
import pyspark 
# iniciamos una sesión spark
sc = pyspark.SparkContext(appName = "ModeloML")

In [3]:
# Para trabajar con DataFrames
from pyspark.sql import SQLContext 
sqlContext = SQLContext(sc)

In [4]:
df = sqlContext.read.options(header='True', inferSchema='True').csv('train.csv')

In [5]:
df.head(4)

[Row(key='2009-06-15 17:26:21.0000001', fare_amount=4.5, pickup_datetime='2009-06-15 17:26:21 UTC', pickup_longitude=-73.844311, pickup_latitude=40.721319, dropoff_longitude=-73.84161, dropoff_latitude=40.712278, passenger_count=1),
 Row(key='2010-01-05 16:52:16.0000002', fare_amount=16.9, pickup_datetime='2010-01-05 16:52:16 UTC', pickup_longitude=-74.016048, pickup_latitude=40.711303, dropoff_longitude=-73.979268, dropoff_latitude=40.782004, passenger_count=1),
 Row(key='2011-08-18 00:35:00.00000049', fare_amount=5.7, pickup_datetime='2011-08-18 00:35:00 UTC', pickup_longitude=-73.982738, pickup_latitude=40.76127, dropoff_longitude=-73.991242, dropoff_latitude=40.750562, passenger_count=2),
 Row(key='2012-04-21 04:30:42.0000001', fare_amount=7.7, pickup_datetime='2012-04-21 04:30:42 UTC', pickup_longitude=-73.98713, pickup_latitude=40.733143, dropoff_longitude=-73.991567, dropoff_latitude=40.758092, passenger_count=1)]

In [6]:
#cantidad de datos
cantidad_total=df.count()
cantidad_total

55423856

In [7]:
# Esquema de los datos
df.printSchema()

root
 |-- key: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)



# Eliminando Valores Nulos de la tabla

In [9]:
# tabla sin valores nulos, sin duplicados
df=df.na.drop().dropDuplicates()
# cantidad de  no valores nulos
cantidad_valores_no_nulos=df.count()
print("cantidad de valores nulos: ",cantidad_total-cantidad_valores_no_nulos)

cantidad de valores nulos:  376


# Estadisticas

In [10]:
summary=df.describe()

In [11]:
# Tipos de datos del summary
summary.dtypes

[('summary', 'string'),
 ('key', 'string'),
 ('fare_amount', 'string'),
 ('pickup_datetime', 'string'),
 ('pickup_longitude', 'string'),
 ('pickup_latitude', 'string'),
 ('dropoff_longitude', 'string'),
 ('dropoff_latitude', 'string'),
 ('passenger_count', 'string')]

In [12]:
summary["summary","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"].show()

+-------+------------------+-----------------+------------------+-----------------+
|summary|  pickup_longitude|  pickup_latitude| dropoff_longitude| dropoff_latitude|
+-------+------------------+-----------------+------------------+-----------------+
|  count|          55423480|         55423480|          55423480|         55423480|
|   mean|-72.50972923276653|39.91981628104406| -72.5112097297193|39.92068144482925|
| stddev|12.848775109035225|9.642324401644247|12.782196517830826|9.633345796415135|
|    min|      -3442.059565|     -3492.263768|      -3442.024565|     -3547.886698|
|    max|       3457.625683|      3408.789565|        3457.62235|      3537.132528|
+-------+------------------+-----------------+------------------+-----------------+



In [13]:
summary["summary","passenger_count","fare_amount"].show()

+-------+------------------+------------------+
|summary|   passenger_count|       fare_amount|
+-------+------------------+------------------+
|  count|          55423480|          55423480|
|   mean|  1.68539135398932|11.345004930221487|
| stddev|1.3276616033280673|20.710866987985728|
|    min|                 0|            -300.0|
|    max|               208|          93963.36|
+-------+------------------+------------------+



In [14]:
# Agrupación por cantidad de pasajeros.
df.groupBy("passenger_count").count().show()

+---------------+--------+
|passenger_count|   count|
+---------------+--------+
|             34|       1|
|              1|38337524|
|              6| 1174647|
|              3| 2432712|
|              5| 3929346|
|              9|      23|
|              4| 1178852|
|              8|       9|
|             49|       1|
|              7|      15|
|             51|       1|
|            129|       2|
|              2| 8175243|
|              0|  195040|
|            208|      64|
+---------------+--------+



observaciones :
1. Valores de passenger_count imposibles , como 34,49,51,129,208.
2. Precios demasiados elevados debido a la cantidad de pasajeros y negativo(imposible).

In [15]:
# cerramos la sesión spark
sc.stop()